## Building A Chatbot
In this notebook We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq



In [2]:
load_dotenv()
## load the GROQ API Key
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Pavan and I am a LLM Engineer")])

AIMessage(content="Hi Pavan, it's nice to meet you! \n\nIt's great to connect with another LLM Engineer.\n\nWhat kind of work are you doing in the field? Are you working on a specific project you're excited about? I'm always eager to learn more about what others are building with large language models.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 23, 'total_tokens': 95, 'completion_time': 0.130909091, 'prompt_time': 0.002122024, 'queue_time': 0.241100193, 'total_time': 0.133031115}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9eba715-2626-4576-b593-cd074ad612dd-0', usage_metadata={'input_tokens': 23, 'output_tokens': 72, 'total_tokens': 95})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Pavan and I am a LLM Engineer"),
        AIMessage(content="Hello Pavan! It's nice to meet you. \n\nAs a LLM Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Pavan, and you are a LLM Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 99, 'total_tokens': 136, 'completion_time': 0.067272727, 'prompt_time': 0.004287977, 'queue_time': 0.238406044, 'total_time': 0.071560704}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dc83e548-7ebc-4817-aadd-f5484b1b3dfc-0', usage_metadata={'input_tokens': 99, 'output_tokens': 37, 'total_tokens': 136})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Pavan and I am a LLM Engineer")],
    config=config
)

In [10]:
response.content

"Hi Pavan,\n\nIt's nice to meet you! That's fascinating. As a large language model, I'm always interested to learn more about the people who build and work with models like me.\n\nWhat kind of work do you do as an LLM Engineer?  Are you working on any cool projects right now?  I'd love to hear more about your experience!\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Pavan.  \n\nI remember that from our earlier conversation! 😊  Is there anything else I can help you with? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 121, 'total_tokens': 153, 'completion_time': 0.058181818, 'prompt_time': 0.006791743, 'queue_time': 0.234197178, 'total_time': 0.064973561}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a8c4ec4a-1f6f-4e9a-82ee-22a007a31e7a-0', usage_metadata={'input_tokens': 121, 'output_tokens': 32, 'total_tokens': 153})

In [13]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content
# The AI does not remember as its a different session id.

"I don't know your name! I am just a language model and don't have access to any personal information about you. 😊\n\nIf you'd like to tell me your name, I'm happy to learn it.  \n\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Hi John! Nice to meet you.  \n\nIs there anything I can help you with today?\n'

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John, remember? 😊  \n\nI'm trying my best to keep track of things!\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Pavan")]})

AIMessage(content="Hello Pavan, nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 32, 'total_tokens': 73, 'completion_time': 0.074545455, 'prompt_time': 0.002544462, 'queue_time': 0.234564137, 'total_time': 0.077089917}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c79f183a-3a7f-478a-8ce6-73ecd18a05bd-0', usage_metadata={'input_tokens': 32, 'output_tokens': 41, 'total_tokens': 73})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [20]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Pavan")],
    config=config
)

response

AIMessage(content="Hi Pavan, it's nice to meet you! \n\nI'm happy to help answer any questions you have. Just ask away! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 32, 'total_tokens': 68, 'completion_time': 0.065454545, 'prompt_time': 0.00237014, 'queue_time': 0.23655086, 'total_time': 0.067824685}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-92d05f37-2260-4a3f-ab34-67d9fdedf96c-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68})

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Pavan.  I remember! 😊  \n\nIs there anything else I can help you with?\n'

In [22]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [23]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Pavan")],"language":"Hindi"})
response.content

'नमस्ते पवन! 👋 \n\nमुझे खुशी है कि आपसे मिलने को मिला। 😊 \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी क्षमता से उत्तर देने की कोशिश करूंगा। \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [24]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Pavan")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष! 😊 \n\nतुम्हें कैसे हो? क्या मैं तुमकी कोई मदद कर सकता हूँ?\n'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [26]:
response.content

'आपका नाम Pavan है। 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [27]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\pavan\Desktop\Simple_RAG\w_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [28]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information or preferences, so I don't know what ice cream you like! \n\nWhat's your favorite flavor? 😊🍦\n"

It cannot remember my favorite ice cream flavor as we have used the trimmer here and that part of the information has beened trimmed

In [29]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2".  \n\n\n\nLet me know if you\'d like to try another one! 😊 \n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"